In [2]:
# --- 1. 환경 설정 셀 (수정된 버전) ---
import os
import sys

# 작업 디렉토리를 올바른 위치로 변경
workspace_path = '/workspace'
os.chdir(workspace_path)

# RFdiffusion 경로를 파이썬이 인식하도록 추가
rfdiffusion_path = os.path.join(workspace_path, 'RFdiffusion')
if rfdiffusion_path not in sys.path:
    sys.path.append(rfdiffusion_path)
import time
import signal
import sys
import random
import string
import re
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
import py3Dmol

# Colab이 아닌 로컬 환경이므로, RFdiffusion 경로를 직접 추가
if 'RFdiffusion' not in sys.path:
    home_dir = os.path.expanduser("~")
    rfdiffusion_path = os.path.join(home_dir, 'RFdiffusion')
    sys.path.append(rfdiffusion_path)

os.environ["DGLBACKEND"] = "pytorch"

from inference.utils import parse_pdb
from colabdesign.rf.utils import get_ca
from colabdesign.rf.utils import fix_contigs, fix_partial_contigs, fix_pdb, sym_it
from colabdesign.shared.protein import pdb_to_string
from colabdesign.shared.plot import plot_pseudo_3D

# --- Colab의 files.upload()를 대체하는 로컬 파일 처리 함수 ---
def get_pdb(pdb_code=None, use_upload=False):
    if use_upload:
        upload_widget = widgets.FileUpload(
            accept='.pdb', description='PDB 파일 업로드', button_style='info'
        )
        display(upload_widget)
        
        def wait_for_upload(widget):
            while len(widget.value) == 0: time.sleep(0.1)
            uploaded_filename = list(widget.value.keys())[0]
            pdb_content = widget.value[uploaded_filename]['content']
            pdb_filename = "tmp.pdb"
            with open(pdb_filename, "wb") as out: out.write(pdb_content)
            print(f"'{uploaded_filename}'이(가) 'tmp.pdb'로 저장되었습니다.")
            return pdb_filename
            
        return wait_for_upload(upload_widget)
    elif pdb_code is None or pdb_code == "":
        print("PDB 코드를 입력하거나 use_upload=True로 설정하세요.")
        return None
    elif os.path.isfile(pdb_code): return pdb_code
    elif len(pdb_code) == 4:
        pdb_filename = f"{pdb_code}.pdb1"
        if not os.path.isfile(pdb_filename):
            os.system(f"wget -qnc https://files.rcsb.org/download/{pdb_code}.pdb1.gz && gunzip -f {pdb_code}.pdb1.gz")
        return pdb_filename
    else:
        pdb_filename = f"AF-{pdb_code}-F1-model_v4.pdb"
        if not os.path.isfile(pdb_filename):
            os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/{pdb_filename}")
        return pdb_filename

def run_ananas(pdb_str, path, sym=None):
    pdb_filename = f"outputs/{path}/ananas_input.pdb"
    out_filename = f"outputs/{path}/ananas.json"
    os.makedirs(f"outputs/{path}", exist_ok=True)
    with open(pdb_filename,"w") as handle: handle.write(pdb_str)
    cmd = f"./ananas {pdb_filename} -u -j {out_filename}"
    if sym is None: os.system(cmd)
    else: os.system(f"{cmd} {sym}")
    try:
        with open(out_filename,"r") as f: out = json.load(f)
        results, AU = out[0], out[-1]["AU"]
        group, chains, rmsd = AU["group"], AU["chain names"], results["Average_RMSD"]
        print(f"AnAnaS detected {group} symmetry at RMSD:{rmsd:.3}")
        C = np.array(results['transforms'][0]['CENTER'])
        A = [np.array(t["AXIS"]) for t in results['transforms']]
        new_lines = []
        for line in pdb_str.split("\n"):
            if line.startswith("ATOM"):
                chain = line[21:22]
                if chain in chains:
                    x = np.array([float(line[i:(i+8)]) for i in [30,38,46]])
                    if group[0] == "c": x = sym_it(x,C,A[0])
                    if group[0] == "d": x = sym_it(x,C,A[1],A[0])
                    coord_str = "".join([f"{a:8.3f}" for a in x])
                    new_lines.append(line[:30]+coord_str+line[54:])
            else: new_lines.append(line)
        return results, "\n".join(new_lines)
    except Exception as e:
        print(f"AnAnaS 결과 처리 중 오류 발생: {e}")
        return None, pdb_str

def run(command, steps, num_designs=1, visual="none"):
    def run_command_and_get_pid(command):
        pid_file = '/dev/shm/pid'
        os.system(f'nohup {command} > /dev/null & echo $! > {pid_file}')
        with open(pid_file, 'r') as f: pid = int(f.read().strip())
        os.remove(pid_file)
        return pid
    def is_process_running(pid):
        try: os.kill(pid, 0)
        except OSError: return False
        else: return True
    run_output = widgets.Output()
    progress = widgets.FloatProgress(min=0, max=1, description='running', bar_style='info')
    display(widgets.VBox([progress, run_output]))
    for n in range(steps):
        if os.path.isfile(f"/dev/shm/{n}.pdb"): os.remove(f"/dev/shm/{n}.pdb")
    pid = run_command_and_get_pid(command)
    try:
        fail = False
        for _ in range(num_designs):
            for n in range(steps):
                wait = True
                while wait and not fail:
                    time.sleep(0.1)
                    if os.path.isfile(f"/dev/shm/{n}.pdb"):
                        with open(f"/dev/shm/{n}.pdb", "r") as f: pdb_str = f.read()
                        if "TER" in pdb_str or "ENDMDL" in pdb_str: wait = False
                        elif not is_process_running(pid): fail = True
                    elif not is_process_running(pid): fail = True
                if fail:
                    progress.bar_style = 'danger'; progress.description = "failed"
                    break
                else:
                    progress.value = (n+1) / steps
                    if visual != "none":
                        with run_output:
                            run_output.clear_output(wait=True)
                            if visual == "image":
                                xyz, bfact = get_ca(f"/dev/shm/{n}.pdb", get_bfact=True)
                                fig = plt.figure(); fig.set_dpi(100);fig.set_figwidth(6);fig.set_figheight(6)
                                ax1 = fig.add_subplot(111);ax1.set_xticks([]);ax1.set_yticks([])
                                plot_pseudo_3D(xyz, c=bfact, cmin=0.5, cmax=0.9, ax=ax1); plt.show()
                            if visual == "interactive":
                                view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
                                view.addModel(pdb_str,'pdb'); view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
                                view.zoomTo(); view.show()
                if os.path.exists(f"/dev/shm/{n}.pdb"): os.remove(f"/dev/shm/{n}.pdb")
            if fail:
                progress.bar_style = 'danger'; progress.description = "failed"
                break
        while is_process_running(pid): time.sleep(0.1)
    except KeyboardInterrupt:
        os.kill(pid, signal.SIGTERM); progress.bar_style = 'danger'; progress.description = "stopped"
    
def run_diffusion(contigs, path, pdb=None, iterations=50, symmetry="none", order=1, hotspot=None,
                  chains=None, add_potential=False, num_designs=1, visual="none"):
    full_path = f"outputs/{path}"; os.makedirs(full_path, exist_ok=True)
    opts = [f"inference.output_prefix={full_path}", f"inference.num_designs={num_designs}"]
    if chains == "": chains = None
    if symmetry in ["auto","cyclic","dihedral"]:
        if symmetry == "auto": sym, copies = None, 1
        else: sym, copies = {"cyclic":(f"c{order}",order), "dihedral":(f"d{order}",order*2)}[symmetry]
    else: symmetry = None; sym, copies = None, 1
    contigs_list = contigs.replace(","," ").replace(":"," ").split()
    is_fixed, is_free = False, False; fixed_chains = []
    for contig in contigs_list:
        for x in contig.split("/"):
            a = x.split("-")[0]
            if a and a[0].isalpha():
                is_fixed = True
                if a[0] not in fixed_chains: fixed_chains.append(a[0])
            if a.isnumeric(): is_free = True
    if len(contigs_list) == 0 or not is_free: mode = "partial"
    elif is_fixed: mode = "fixed"
    else: mode = "free"
    if mode in ["partial","fixed"]:
        pdb_str = pdb_to_string(pdb, chains=chains)
        if symmetry == "auto":
            a, pdb_str = run_ananas(pdb_str, path)
            if a is None: print(f'ERROR: no symmetry detected'); symmetry = None; sym, copies = None, 1
            else:
                if a["group"][0] == "c": symmetry = "cyclic"; sym, copies = a["group"], int(a["group"][1:])
                elif a["group"][0] == "d": symmetry = "dihedral"; sym, copies = a["group"], 2 * int(a["group"][1:])
                else: print(f'ERROR: detected symm ({a["group"]}) not supported'); symmetry = None; sym, copies = None, 1
        elif mode == "fixed": pdb_str = pdb_to_string(pdb_str, chains=fixed_chains)
        pdb_filename = f"{full_path}/input.pdb"
        with open(pdb_filename, "w") as handle: handle.write(pdb_str)
        parsed_pdb = parse_pdb(pdb_filename)
        opts.append(f"inference.input_pdb={pdb_filename}")
        if mode in ["partial"]:
            iterations = int(80 * (iterations / 200)); opts.append(f"diffuser.partial_T={iterations}")
            contigs_list = fix_partial_contigs(contigs_list, parsed_pdb)
        else: opts.append(f"diffuser.T={iterations}"); contigs_list = fix_contigs(contigs_list, parsed_pdb)
    else:
        opts.append(f"diffuser.T={iterations}"); parsed_pdb = None
        contigs_list = fix_contigs(contigs_list, parsed_pdb)
    if hotspot is not None and hotspot != "": opts.append(f"ppi.hotspot_res=[{hotspot}]")
    if sym is not None:
        sym_opts = ["--config-name symmetry", f"inference.symmetry={sym}"]
        if add_potential: sym_opts += ["'potentials.guiding_potentials=[\"type:olig_contacts,weight_intra:1,weight_inter:0.1\"]'", "potentials.olig_intra_all=True","potentials.olig_inter_all=True", "potentials.guide_scale=2","potentials.guide_decay=quadratic"]
        opts = sym_opts + opts; contigs_list = sum([contigs_list] * copies,[])
    opts.append(f"'contigmap.contigs=[{' '.join(contigs_list)}]'")
    opts += ["inference.dump_pdb=True","inference.dump_pdb_path='/dev/shm'"]
    print("mode:", mode); print("output:", full_path); print("contigs:", contigs_list)
    run_script_path = "/workspace/RFdiffusion/run_inference.py"
    opts_str = " ".join(opts)
    cmd = f"python {run_script_path} {opts_str}"
    print(cmd)
    run(cmd, iterations, num_designs, visual=visual)
    for n in range(num_designs):
        pdbs = [f"outputs/traj/{path}_{n}_pX0_traj.pdb", f"outputs/traj/{path}_{n}_Xt-1_traj.pdb", f"{full_path}_{n}.pdb"]
        for pdb_file in pdbs:
            if os.path.exists(pdb_file):
                with open(pdb_file,"r") as handle: pdb_str = handle.read()
                with open(pdb_file,"w") as handle: handle.write(fix_pdb(pdb_str, contigs_list))
    return contigs_list, copies

print("✅ Setup cell is ready.")

✅ Setup cell is ready.


In [ ]:
%%time
import os
import sys
import pandas as pd
import glob

# --- 1. 설정 및 파라미터 입력 부분 (첫 번째 코드) ---

# @markdown ### **👇 여기에 처리할 폴더와 Contig 정보를 입력하세요.**
# @markdown ---
input_folder = "outputs/human-best"  # @param {type:"string"}
contig_for_all_pdbs = "C311-391/0 100-100"  # @param {type:"string"}

# @markdown ---
# @markdown ### **MPNN 및 AlphaFold 공통 설정**
# @markdown 이 설정은 위 폴더의 모든 PDB 파일에 동일하게 적용됩니다.
num_seqs = 8       #@param {type:"raw"}
num_recycles = 3    #@param {type:"raw"}
initial_guess = True
use_multimer = True
rm_aa = "C"
mpnn_sampling_temp = 0.1

# --- 2. 개별 PDB 파일 처리 루프 (첫 번째 코드) ---

print(f"🚀 Settings: num_seqs={num_seqs}, num_recycles={num_recycles}")
pdb_files = sorted(glob.glob(os.path.join(input_folder, '*.pdb')))

all_results_list = [] # 모든 결과를 저장할 리스트

if not pdb_files:
    print(f"🚨 에러: '{input_folder}' 폴더에서 PDB 파일을 찾을 수 없습니다.")
else:
    print(f"총 {len(pdb_files)}개의 PDB 파일을 대상으로 작업을 시작합니다.")
    print("="*60)

    for i, pdb_path in enumerate(pdb_files):
        print(f"\n({i+1}/{len(pdb_files)}) 번째 작업 실행 중...")
        print(f"  - 입력 파일: {pdb_path}")

        pdb_basename = os.path.basename(pdb_path).replace('.pdb', '')
        output_dir_name = f"{pdb_basename}_mpnn_results"
        output_loc = os.path.join(input_folder, output_dir_name)
        os.makedirs(output_loc, exist_ok=True)
        print(f"  - 결과 저장: {output_loc}")

        contigs_str = ":".join(contig_for_all_pdbs.replace("'", "").replace('"', '').split())
        opts = [f"--pdb={pdb_path}", f"--loc={output_loc}", f"--contig={contigs_str}",
                f"--copies=1", f"--num_seqs={num_seqs}", f"--num_recycles={num_recycles}",
                f"--rm_aa={rm_aa}", f"--mpnn_sampling_temp={mpnn_sampling_temp}", f"--num_designs=1"]
        if initial_guess: opts.append("--initial_guess")
        if use_multimer: opts.append("--use_multimer")
        opts_str = ' '.join(opts)

        # ColabDesign 실행
        get_ipython().system(f"python -m colabdesign.rf.designability_test {opts_str}")
        
        results_csv_path = f"{output_loc}/mpnn_results.csv"
        
        if os.path.exists(results_csv_path):
            df = pd.read_csv(results_csv_path)
            best_row = df.sort_values(by=['design', 'n'], ascending=True).iloc[0]
            rmsd_val = best_row['rmsd']
            plddt_val = best_row['plddt']
            print(f"  ✅ 완료! Best RMSD: {rmsd_val:.4f}, pLDDT: {plddt_val:.2f}")

            # 나중에 합치기 위해 원본 파일명 열 추가
            df['source_pdb'] = os.path.basename(pdb_path)
            all_results_list.append(df)
        else:
            print(f"  ⚠️ 경고: 최종 결과 파일({results_csv_path})이 생성되지 않았습니다.")
        print("-" * 60)

# --- 3. 최종 결과 취합 및 요약 (두 번째 코드) ---

print("\n🎉 모든 개별 작업이 완료되었습니다. 전체 결과를 요약합니다.")
print("="*60)

if all_results_list:
    final_df = pd.concat(all_results_list, ignore_index=True)
    final_df_sorted = final_df.sort_values(by="rmsd", ascending=True)

    # --- ✨ 최종 종합 결과 파일로 저장 ✨ ---
    # (1) 최종 CSV 파일로 저장
    final_csv_path = os.path.join(input_folder, "mpnn_results.csv")
    final_df_sorted.to_csv(final_csv_path, index=False)
    print(f"✅ 최종 종합 결과가 CSV 파일로 저장되었습니다: {final_csv_path}")

    print("\n📊 **전체 디자인 상세 결과 (RMSD 오름차순)**")
    
    # i_pae가 있는지 확인하고 출력할 컬럼 결정
    display_cols = ['source_pdb', 'n', 'plddt', 'rmsd']
    if 'i_pae' in final_df_sorted.columns:
        display_cols.insert(3, 'i_pae')
    
    # .to_string()을 이용해 전체 데이터프레임 출력
    print(final_df_sorted[display_cols].to_string(index=False))
    
    print("-" * 60)
    overall_best = final_df_sorted.iloc[0]
    print(f"🏆 **최종 BEST 디자인** 🏆")
    print(f"  - 원본 백본 파일: {overall_best['source_pdb']}")
    print(f"  - 시퀀스 번호 (n): {overall_best['n']}")
    print(f"  - **최종 RMSD: {overall_best['rmsd']:.4f}**")
    print(f"  - 최종 pLDDT: {overall_best['plddt']:.2f}")
    
    # i_pae 값이 존재하고 NaN이 아닐 경우에만 출력
    if 'i_pae' in overall_best and not pd.isna(overall_best['i_pae']):
        print(f"  - 최종 i_pae: {overall_best['i_pae']:.2f}")
    
    best_pdb_loc_name = overall_best['source_pdb'].replace('.pdb','_mpnn_results')
    best_pdb_loc_path = os.path.join(input_folder, best_pdb_loc_name)
    print(f"\n  👉 이 BEST 디자인의 PDB 파일은 '{best_pdb_loc_path}/best.pdb' 에서 확인할 수 있습니다.")

else:
    print("처리된 결과가 없어 최종 요약을 생성할 수 없습니다.")

🚀 Settings: num_seqs=8, num_recycles=3
총 68개의 PDB 파일을 대상으로 작업을 시작합니다.

(1/68) 번째 작업 실행 중...
  - 입력 파일: outputs/human-best/0806-150-300_5m3ce_168.pdb
  - 결과 저장: outputs/human-best/0806-150-300_5m3ce_168_mpnn_results
{'pdb':'outputs/human-best/0806-150-300_5m3ce_168.pdb','loc':'outputs/human-best/0806-150-300_5m3ce_168_mpnn_results','contigs':'C311-391/0:100-100','copies':1,'num_seqs':8,'initial_guess':True,'use_multimer':True,'use_soluble':False,'num_recycles':3,'rm_aa':'C','num_designs':1,'mpnn_sampling_temp':0.1}
protocol=binder
2025-09-09 05:20:05.120248: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.8 which is older than the ptxas CUDA version (12.9.86). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
running proteinMPNN...
running AlphaFold...
design:0 n:0 mpnn:

design:0 n:5 mpnn:1.201 plddt:0.927 i_ptm:0.649 i_pae:9.477 rmsd:5.078 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/GVKPEELKKFTEELKKRLKERTAEENAEERAAAEAAFAAELAARLAAAPASEHAAVEAEAAKRWAALQAELEKRAAAREAQTEKTIAEIEKNTEKRIKEL
design:0 n:6 mpnn:1.209 plddt:0.822 i_ptm:0.113 i_pae:24.062 rmsd:22.622 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/GVKPEEIKEENEKLQKRRKERVDEEIAARRAALEAEFAAELAAALAAAPPERHAAVLAEAERRRAERLAALAAEAAALRAQAEKINEEINKNTEKYIESL
design:0 n:7 mpnn:1.201 plddt:0.806 i_ptm:0.090 i_pae:25.053 rmsd:40.971 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/GIKPEELKEETEKLKKRLKERVEEEAKAKREKREAEFAAKFAARMAAAPPADHAAVKAEAAKERAALQAELEKEEKKLKEQTEKTIEEIEKNTKKRIESL
  ✅ 완료! Best RMSD: 5.5263, pLDDT: 0.86
------------------------------------------------------------

(4/68) 번째 작업 실행 중...
  - 입력 파일: outputs/human-best/0807-150-600_418.pdb
  - 결과 저장: outputs/human-best/0807-150-600_418_mpnn_results
{

design:0 n:2 mpnn:1.207 plddt:0.886 i_ptm:0.059 i_pae:28.085 rmsd:67.879 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/MEELEKLKEKKREEEEKKRREEAKKQNEENERRIEERKKEEEKKRKELEKKREKLEAEREEVEKKLEEMKENKEEYSEEEIEKLEKELEKLKKEIEELKK
design:0 n:3 mpnn:1.148 plddt:0.884 i_ptm:0.356 i_pae:19.521 rmsd:6.436 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/ELEEKKRKEEERKKKEEERRKRAEEQNERNRRRREEEERRRKEEEEKLKKKKEELKKEAKKVEKELEEMKKNKEKYSEEEIKKLEEELKKLKAEIEKLEA
design:0 n:4 mpnn:1.242 plddt:0.935 i_ptm:0.678 i_pae:10.776 rmsd:2.891 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SLKEELEKEEERKKKEEERRKKAEEQNKRNEELKAEKEKAEAEERAKLEEKRAELKARAEEVRRKLAEMEEHREEYSEEEIERLREELERLEAEVARLEA
design:0 n:5 mpnn:1.148 plddt:0.889 i_ptm:0.517 i_pae:16.116 rmsd:2.730 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/KEEEEKKKEEEREKKEKERREKAEKQNKENEERKKRLEEEEEKKRKELEKEKKELEKKAEEVKKKLEEMEKNKEKYSKEE

running proteinMPNN...
running AlphaFold...
design:0 n:0 mpnn:1.171 plddt:0.859 i_ptm:0.106 i_pae:24.423 rmsd:34.638 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SEEEVKKISEELEKKEKELQEKSKKIKEELKKLSKEEREEVEKAIEEMEETGADSEEYKEILEKYREEGNEEAANILETARERIKEREKFREELKKKLKE
design:0 n:1 mpnn:1.194 plddt:0.912 i_ptm:0.667 i_pae:8.542 rmsd:3.883 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SKEEVEELSKELEEKEKELQEKSKEIKEKLAKFSPEVRAEVEAVIERMEATGADDPAYLAVREQYKAEGNEEVAEVLEKAREREKEREEFRKELRKKLKE
design:0 n:2 mpnn:1.217 plddt:0.778 i_ptm:0.100 i_pae:24.110 rmsd:25.727 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SKEEVEELGKLIEEKEKKLQEEGKKIKEALAKLSPETRAKILAIMDETEATGADSPESEAVRAQYEAEGNKEAAEILARHIKRIKDREEFRKELRKKLKE
design:0 n:3 mpnn:1.235 plddt:0.898 i_ptm:0.625 i_pae:9.343 rmsd:2.499 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/TAEEVKKRGEEIEAKQKELQAKSKEMKEKLAKLSPEE

{'pdb':'outputs/human-best/0811-150-200_24.pdb','loc':'outputs/human-best/0811-150-200_24_mpnn_results','contigs':'C311-391/0:100-100','copies':1,'num_seqs':8,'initial_guess':True,'use_multimer':True,'use_soluble':False,'num_recycles':3,'rm_aa':'C','num_designs':1,'mpnn_sampling_temp':0.1}
protocol=binder
2025-09-09 05:28:51.380106: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.8 which is older than the ptxas CUDA version (12.9.86). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
running proteinMPNN...
running AlphaFold...
design:0 n:0 mpnn:1.039 plddt:0.863 i_ptm:0.064 i_pae:26.535 rmsd:29.028 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SLEEKKKKIKELLEKANKLYNEGGDPEEYKKAYEEAKELLEELKPGAKELLDEYRKLKEEVDELKKKYLETKDAEVKNKLLEAKKKLEEVKK

...

running proteinMPNN...
running AlphaFold...
design:0 n:0 mpnn:1.175 plddt:0.743 i_ptm:0.597 i_pae:15.083 rmsd:7.229 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/ELTAEQKAQIEAMIAALAAADERAAAAAAREAELLAQAAKNPADYAAIAAAAAAARAALAAALAAERAAALAAAKALGLSENELRYRLLESRVARKKEEA
design:0 n:1 mpnn:1.021 plddt:0.800 i_ptm:0.079 i_pae:26.252 rmsd:34.655 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/MLTEEEKKKIEEMIEKLLKLEEELKKAKEEYEKKLKEAEKNPENYEKIAKKLKEELEEKKKKAEEEKEKIYKEAKELGLSKNELKYKLIKKKVEKKKKEK
design:0 n:2 mpnn:1.082 plddt:0.747 i_ptm:0.061 i_pae:27.016 rmsd:41.603 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/ELTEEEKKKIEEMIEELIEIEKKLKEAEEKLEKELAEAKKKPETYEEIAKKLKAEYEKLKKEAEEKKKKLLEEAKKLGLSENELKSKLREKKVKKKKEES
design:0 n:3 mpnn:1.098 plddt:0.801 i_ptm:0.096 i_pae:25.187 rmsd:33.908 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/MLTEEQKKKIEEMIEKILEIEKEKEEAEEELKKK

{'pdb':'outputs/human-best/0906-150-600sc_18.pdb','loc':'outputs/human-best/0906-150-600sc_18_mpnn_results','contigs':'C311-391/0:100-100','copies':1,'num_seqs':8,'initial_guess':True,'use_multimer':True,'use_soluble':False,'num_recycles':3,'rm_aa':'C','num_designs':1,'mpnn_sampling_temp':0.1}
protocol=binder
2025-09-09 06:04:07.850299: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.8 which is older than the ptxas CUDA version (12.9.86). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
running proteinMPNN...
running AlphaFold...
design:0 n:0 mpnn:1.097 plddt:0.817 i_ptm:0.275 i_pae:18.026 rmsd:33.904 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/NLVEKLIKLLEERFEETKKEIEEKIKETLAKLKEPSEEVEKLLKEIEEKIKELEEEKEKVIKEMKENKDPDKNVKLYFEYLEKLKELQ

design:0 n:6 mpnn:1.225 plddt:0.782 i_ptm:0.727 i_pae:11.256 rmsd:11.112 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SMEERYREVLERAKKSNEKNEQTKKQIAEAKKLKAEGTALKAKGEATGDKELLAEGEAKLAKANALFAKLSTGTTQANIAAEAAAAAAAALASLLLLLLL
design:0 n:7 mpnn:1.022 plddt:0.868 i_ptm:0.698 i_pae:9.894 rmsd:2.309 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SMEKRYEEVKKRAEKSNALNAEAKEQIKKAEELKAAGEKAIAEGKKAGDKEKIAEGKEKLAEAEALEKKLATKTTKDNIKAEKAAAAAAAALAAAAAALL
  ✅ 완료! Best RMSD: 2.9136, pLDDT: 0.86
------------------------------------------------------------

(59/68) 번째 작업 실행 중...
  - 입력 파일: outputs/human-best/0906-150-600sc_502.pdb
  - 결과 저장: outputs/human-best/0906-150-600sc_502_mpnn_results
{'pdb':'outputs/human-best/0906-150-600sc_502.pdb','loc':'outputs/human-best/0906-150-600sc_502_mpnn_results','contigs':'C311-391/0:100-100','copies':1,'num_seqs':8,'initial_guess':True,'use_multimer':True,'use_soluble':False,'num_recycles':3,'rm_aa':'

design:0 n:3 mpnn:1.124 plddt:0.841 i_ptm:0.101 i_pae:24.665 rmsd:32.129 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/GAIDRVAEKEAAAFAAAEKKAKEALAKAGEDPALLEKVAAELDAEAKAKKATAANIKAARPLTPYVAAYLAVLNAAAAKYAALAKALKDAGAAWKAEKAA
design:0 n:4 mpnn:0.987 plddt:0.806 i_ptm:0.112 i_pae:24.057 rmsd:36.595 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SYIEKKKKEIEEELKKYEKELEKAKKEAEKNPEKLKKKIEELKKKAKHYRDTAKRIREARPLTPEVEAYLKVLEEAAKKYEKLAKEAEEYYEKLKKEKEK
design:0 n:5 mpnn:1.113 plddt:0.908 i_ptm:0.558 i_pae:11.344 rmsd:0.999 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SYIEKKKAKEEAELAKLEKEAAAARAALPADPAAAVEKQAELQKLSKHYNNTAANIKAAQPLTPEVQAYIDVLNDAAKKYQALADELAADAAAWAAAQAA
design:0 n:6 mpnn:1.089 plddt:0.932 i_ptm:0.634 i_pae:9.743 rmsd:0.925 GFINIFPMINTTVFVNDGENVDLIVEYEAFPKPEHQQWIYMNRTFTDKWEDYPKSENESNIRYVSELHLTRLKGTEGGTYT/SFIKRKREKLEKELEEKEKEAEEAIEKLPEDPELLKKKIEELRALSRHYRRTAEDIRAASPLTPEVEAYLKVLEEAAERY

running proteinMPNN...
running AlphaFold...


In [ ]:
# --- Top n 디자인을 선별하고 PDB 파일을 생성하는 셀 --- 기본 판별 코드(sc x)

import pandas as pd
import os
import shutil

# 1. 결과 파일 및 경로 설정 (이 부분은 이미 올바르게 수정되었습니다)
results_csv_path = "outputs/0907-50-600/0907-50-600/mpnn_results.csv"
source_pdb_folder = "outputs/0907-50-600/0907-50-600"
ranked_pdb_folder = "outputs/0907-50-600/0907-50-600/ranked_top_100"

print(f"'{results_csv_path}' 파일에서 결과 분석을 시작합니다.")

# 2. 결과 파일이 있는지 확인
if not os.path.exists(results_csv_path):
    print(f" 에러: 결과 파일({results_csv_path})을 찾을 수 없습니다. 이전 셀이 성공적으로 실행되었는지 확인하세요.")
else:
    # 3. CSV 파일 읽어오기
    df_all = pd.read_csv(results_csv_path)
    
    # 4. i_pae < 10 조건으로 필터링
    #    'i_pae' 컬럼이 존재하므로 그대로 사용합니다.
    if 'i_pae' in df_all.columns:
        df_filtered = df_all[df_all['i_pae'] < 15].copy()
        print(f"i_pae < 10 조건을 만족하는 디자인 {len(df_filtered)}개를 찾았습니다.")
    else:
        print(" 경고: 'i_pae' 컬럼을 찾을 수 없습니다. 필터링 없이 모든 디자인으로 진행합니다.")
        df_filtered = df_all.copy()

    # 5. RMSD 기준으로 오름차순 정렬
    df_sorted = df_filtered.sort_values(by='rmsd', ascending=True)

    # 6. 상위 100개 디자인 선택
    top_100_designs = df_sorted.head(100)
    print(f"RMSD가 낮은 순서대로 상위 {len(top_100_designs)}개를 선별했습니다.")

    # 7. 순위별 PDB 파일을 저장할 폴더 생성
    os.makedirs(ranked_pdb_folder, exist_ok=True)
    print(f"선별된 PDB 파일은 '{ranked_pdb_folder}' 폴더에 저장됩니다.")

    # 8. PDB 파일 복사 및 이름 변경
    copied_count = 0
    for rank, (index, row) in enumerate(top_100_designs.iterrows(), 1):
        # 'design'과 'n' 컬럼을 조합하여 파일명 생성
        design_name = f"design{row['design']}_n{row['n']}"  # 이 부분을 수정
        source_pdb_file = f"{source_pdb_folder}/{design_name}.pdb"
        
        # 순위를 포함한 새로운 파일명 지정
        rmsd_val = row['rmsd']
        dest_pdb_file = f"{ranked_pdb_folder}/rank_{rank:03d}_{design_name}_rmsd_{rmsd_val:.2f}.pdb"

        if os.path.exists(source_pdb_file):
            shutil.copy(source_pdb_file, dest_pdb_file)
            copied_count += 1
        else:
            print(f"  - 경고: 원본 파일 '{source_pdb_file}'을 찾을 수 없습니다.")

    print(f"\n✅ 작업 완료: 총 {copied_count}개의 PDB 파일을 순위별로 저장했습니다.")
    
    # 9. 선별된 Top 100 디자인의 통계 정보 출력
    # 'name' 대신 'plddt', 'i_pae', 'rmsd' 컬럼을 출력합니다. 'design'과 'n'도 포함합니다.
    print("\n📊 선별된 Top 100 디자인 목록:")
    print(top_100_designs[['design', 'n', 'plddt', 'i_pae', 'rmsd']].to_string(index=False)) # 이 부분을 수정

In [ ]:
# --- 4번 셀: Top N 디자인 선별 및 PDB 파일 저장 --- sc 판별 코드

import pandas as pd
import os
import shutil

# @markdown ### **👇 분석할 실행 정보와 필터링 조건을 입력하세요.**
# @markdown ---

# <<< (1) 3번 셀에서 사용했던 `run_name`을 정확히 입력하세요.
run_name = "0908-150-600sc"  # @param {type:"string"}

# <<< (2) 몇 개의 상위 디자인을 선별할지 숫자를 입력하세요.
top_n = 20  # @param {type:"integer"}

# <<< (3) PAE 점수 필터링 기준값을 입력하세요. (낮을수록 좋음)
pae_threshold = 15.0  # @param {type:"number"}

# --- 아래는 수정할 필요 없이 그대로 실행하면 됩니다. ---

# 1. 결과 파일 및 경로 설정
output_dir = f"outputs/{run_name}"
# ✨ 3번 셀에서 생성된 우리 결과 파일을 사용합니다.
results_csv_path = os.path.join(output_dir, "all_results_parsed.csv") 
ranked_pdb_folder = os.path.join(output_dir, f"ranked_top_{top_n}")

print(f"'{results_csv_path}' 파일에서 결과 분석을 시작합니다.")

# 2. 결과 파일이 있는지 확인
if not os.path.exists(results_csv_path):
    print(f"🚨 에러: 결과 파일({results_csv_path})을 찾을 수 없습니다. run_name을 확인하거나 3번 셀을 먼저 실행하세요.")
else:
    # 3. CSV 파일 읽어오기
    df_all = pd.read_csv(results_csv_path)
    
    # 4. PAE 점수 조건으로 필터링
    # ✨ 'i_pae' 대신 우리 결과 파일에 있는 'pae' 컬럼을 사용합니다.
    if 'pae' in df_all.columns:
        df_filtered = df_all[df_all['pae'] < pae_threshold].copy()
        print(f"PAE < {pae_threshold} 조건을 만족하는 디자인 {len(df_filtered)}개를 찾았습니다.")
    else:
        print("⚠️ 경고: 'pae' 컬럼을 찾을 수 없습니다. 필터링 없이 진행합니다.")
        df_filtered = df_all.copy()

    # 5. RMSD 기준으로 오름차순 정렬 (낮을수록 좋음)
    df_sorted = df_filtered.sort_values(by='rmsd', ascending=True)

    # 6. 상위 N개 디자인 선택
    top_designs = df_sorted.head(top_n)
    print(f"RMSD가 낮은 순서대로 상위 {len(top_designs)}개를 선별했습니다.")

    # 7. 순위별 PDB 파일을 저장할 폴더 생성
    os.makedirs(ranked_pdb_folder, exist_ok=True)
    print(f"선별된 PDB 파일은 '{ranked_pdb_folder}' 폴더에 저장됩니다.")

    # 8. PDB 파일 복사 및 이름 변경
    copied_count = 0
    for rank, (index, row) in enumerate(top_designs.iterrows(), 1):
        design_num = int(row['design'])
        seq_num = int(row['n'])
        
        # ✨ ColabDesign이 생성한 최종 PDB 파일명 형식에 맞게 경로를 지정합니다.
        source_pdb_file = os.path.join(output_dir, f"design_{design_num}_n{seq_num}.pdb")
        
        # 순위와 주요 점수를 포함한 새로운 파일명 지정
        rmsd_val = row['rmsd']
        plddt_val = row['plddt']
        dest_pdb_file = os.path.join(ranked_pdb_folder, f"rank_{rank:03d}_design{design_num}n{seq_num}_plddt{plddt_val:.2f}_rmsd{rmsd_val:.2f}.pdb")

        if os.path.exists(source_pdb_file):
            shutil.copy(source_pdb_file, dest_pdb_file)
            copied_count += 1
        else:
            # ✨ designability_test 스크립트는 모든 PDB를 저장하지 않을 수 있으므로, 파일이 없는 경우는 일반적인 경고로 처리합니다.
            # print(f"   - 정보: 원본 파일 '{source_pdb_file}'은(는) 생성되지 않아 건너뜁니다.")
            pass

    print(f"\n✅ 작업 완료: 총 {copied_count}개의 PDB 파일을 순위별로 저장했습니다.")
    
    # 9. 선별된 디자인의 통계 정보 출력
    # ✨ 우리 결과 파일에 있는 컬럼 이름('pae')으로 수정합니다.
    print(f"\n📊 선별된 Top {len(top_designs)} 디자인 목록:")
    print(top_designs[['design', 'n', 'plddt', 'pae', 'rmsd']].to_string(index=False))